<a href="https://colab.research.google.com/github/Bennykillua/rag_with_milvus_and_llamaindex.ipynb/blob/main/RAG_with_milvus_and_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/tutorials/integration/rag_with_milvus_and_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>   <a href="https://github.com/Bennykillua/rag_with_milvus_and_llamaindex.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/View%20on%20GitHub-555555?style=flat&logo=github&logoColor=white" alt="GitHub Repository"/>
</a>

# Retrieval-Augmented Generation (RAG) with Milvus and LlamaIndex

This guide demonstrates how to build a Retrieval-Augmented Generation (RAG) system using LlamaIndex and Milvus.

Milvus is an open-source vector database for storing, processing, running, indexes, and retrieving vector embedding across various environments.

LlamaIndex is an orchestration framework that simplifies building LLM applications by integrating private, domain-specific, and public data.

## Before you begin

### Install dependencies
Code snippets on this page require pymilvus and llamaindex dependencies. You can install them using the following commands:

In [ ]:
%pip install pymilvus>=2.4.2

In [ ]:
%pip install llama-index-vector-stores-milvus

In [ ]:
%pip install llama-index

> If you are using Google Colab, to enable dependencies just installed, you may need to **restart the runtime** (click on the "Runtime" menu at the top of the screen, and select "Restart session" from the dropdown menu).

### Setup OpenAI

Lets first begin by adding the openai api key. This will allow us to access chatgpt.

In [ ]:
import openai

openai.api_key = "OpenAI-API-Key"

### Prepare data

You can download sample data with the following commands:

In [ ]:
! mkdir -p 'data/'
! wget 'https://raw.githubusercontent.com/dbamman/litbank/refs/heads/master/original/730_oliver_twist.txt' -O 'data/730_oliver_twist.txt'
! wget 'https://raw.githubusercontent.com/dbamman/litbank/refs/heads/master/original/932_the_fall_of_the_house_of_usher.txt' -O 'data/932_the_fall_of_the_house_of_usher.txt'

--2024-10-11 09:16:28--  https://raw.githubusercontent.com/dbamman/litbank/refs/heads/master/original/730_oliver_twist.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 892131 (871K) [text/plain]
Saving to: ‘data/730_oliver_twist.txt’

data/730_oliver_twi 100%[===================>] 871.22K  --.-KB/s    in 0.06s   

2024-10-11 09:16:28 (14.7 MB/s) - ‘data/730_oliver_twist.txt’ saved [892131/892131]

--2024-10-11 09:16:28--  https://raw.githubusercontent.com/dbamman/litbank/refs/heads/master/original/932_the_fall_of_the_house_of_usher.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected

## Getting Started

### Generate our data
Then, you will generate a document from the `data/730_oliver_twist.txt` novel using a `SimpleDirectoryReader` class from lama_index library.


In [ ]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(
    input_files=["data/730_oliver_twist.txt"]
).load_data()

print("Document ID:", documents[0].doc_id)

Document ID: 75698388-17c3-4eb5-8bb0-4e32c90b9d3e


### Create an index across the data

Now that we have a document, we can can create an index and insert the document.

> Please note that **Milvus Lite** requires `pymilvus>=2.4.2`.

In [ ]:
# Create an index over the documents
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore


vector_store = MilvusVectorStore(uri="./milvus_demo.db", dim=1536, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 2680fe285a7a46bfa23a442251e3ffd1
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection


> For the parameters of `MilvusVectorStore`:
> - Setting the `uri` as a local file, e.g.`./milvus.db`, is the most convenient method, as it automatically utilizes [Milvus Lite](https://milvus.io/docs/milvus_lite.md) to store all data in this file.
> - If you have large scale of data, you can set up a more performant Milvus server on [docker or kubernetes](https://milvus.io/docs/quickstart.md). In this setup, please use the server uri, e.g.`http://localhost:19530`, as your `uri`.
> - If you want to use [Zilliz Cloud](https://zilliz.com/cloud), the fully managed cloud service for Milvus, adjust the `uri` and `token`, which correspond to the [Public Endpoint and Api key](https://docs.zilliz.com/docs/on-zilliz-cloud-console#free-cluster-details) in Zilliz Cloud.

### Query the data
Now that we have our document stored in the index, we can ask questions against the index. The index will use the data stored in itself as the knowledge base for chatgpt.

In [ ]:
query_engine = index.as_query_engine()
res = query_engine.query("how did Oliver twist grow up?")
print(res)

Oliver Twist grew up in a workhouse where he was a victim of treachery and deception. He was eventually dispatched to a branch-workhouse where he was under the care of an elderly female who provided minimal nourishment and care. Despite the harsh conditions and lack of proper care, Oliver displayed a sturdy spirit that helped him survive his difficult upbringing.


In [ ]:
res = query_engine.query("What motivates Oliver to ask for more food in the workhouse?")
print(res)

Oliver is motivated to ask for more food in the workhouse due to his extreme hunger and desperation.


This next test shows that overwriting removes the previous data.

In [ ]:
from llama_index.core import Document


vector_store = MilvusVectorStore(uri="./milvus_demo.db", dim=1536, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    [Document(text="The number that is being searched for is ten.")],
    storage_context,
)
query_engine = index.as_query_engine()
res = query_engine.query("how did Oliver twist grow up?")
print(res)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: f8c2566a84f94f99852648728faf8ef0
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection


Oliver Twist grew up in a workhouse and later on the streets of London, where he faced many challenges and hardships.


The next test shows adding additional data to an already existing  index.

In [ ]:
del index, vector_store, storage_context, query_engine

vector_store = MilvusVectorStore(uri="./milvus_demo.db", overwrite=False)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
query_engine = index.as_query_engine()
res = query_engine.query("What is the number?")
print(res)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: b16d3124a3c345c9a7d3e894ddac089f


The number is ten.


In [ ]:
res = query_engine.query("how did Oliver twist grow up?")
print(res)

Oliver Twist grew up in a workhouse where he was a victim of treachery and deception. He was poorly fed and clothed, and was eventually sent to a branch-workhouse where he was under the care of an elderly woman who prioritized her own needs over the children's welfare. Oliver's upbringing was marked by neglect, hunger, and harsh conditions, leading to him being a pale, thin child with a small stature. Despite these challenges, Oliver possessed a resilient spirit that helped him survive his difficult circumstances.


## Metadata filtering

We can generate results by filtering specific sources. The following example illustrates loading all documents from the directory and subsequently filtering them based on metadata.

In [ ]:
from llama_index.core.vector_stores import ExactMatchFilter, MetadataFilters

# Load all the two documents loaded before
documents_all = SimpleDirectoryReader("./data/").load_data()

vector_store = MilvusVectorStore(uri="./milvus_demo.db", dim=1536, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents_all, storage_context)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: f6bafafe6c9847a3855eb0532f95b5e0
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection


We want to only retrieve documents from the file `932_the_fall_of_the_house_of_usher.txt`.

In [ ]:
filters = MetadataFilters(
    filters=[ExactMatchFilter(key="file_name", value="932_the_fall_of_the_house_of_usher.txt")]
)
query_engine = index.as_query_engine(filters=filters)
res = query_engine.query("What distinctive physical feature does Roderick Usher exhibit in The Fall of the House of Usher?")

print(res)

Roderick Usher exhibits distinctive physical features such as a cadaverous complexion, large, liquid, and luminous eyes, thin and pallid lips with a beautiful curve, a delicate Hebrew model nose with broad nostrils, a finely-moulded chin lacking prominence, and hair of exceptional softness and thinness.


We get a different result this time when retrieve from the file `paul_graham_essay.txt`.

In [ ]:
filters = MetadataFilters(
    filters=[ExactMatchFilter(key="file_name", value="730_oliver_twist.txt")]
)
query_engine = index.as_query_engine(filters=filters)
res = query_engine.query("What challenges did Oliver face?")

print(res)

Oliver faced challenges such as sore feet, weak legs, fatigue, being ignored and mistreated by people, fear of being sent to jail for begging, being threatened by farmers and shopkeepers, and experiencing loneliness and desolation.
